In [3]:
from datasets import load_from_disk
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from PIL import Image  # Importar PIL para manejar imágenes
import os  # Importar os para verificar rutas de archivos

# ----------------------------
# Parámetros
# ----------------------------
_checkpoint = 'VT_V8/checkpoint-9065'  # Ruta al checkpoint a cargar
_dataset_test = '/Volumes/m2/Memoria/Code/PMM/BeningMalignant_Masses-Preprocessing/data/processed/subset_datos/inbreastmiasvinddsm_arrowsplit/test'  # Ruta al dataset de prueba

# ----------------------------
# Cargar el dataset desde el disco en formato Arrow
# ----------------------------
dataset = load_from_disk(_dataset_test)

# ----------------------------
# Obtener las etiquetas desde las características del dataset
# ----------------------------
labels = dataset.features['label'].names  # ['Benign Masses', 'Malignant Masses']

# Crear diccionarios de mapeo entre etiquetas y IDs
label2id = {label: idx for idx, label in enumerate(labels)}
id2label = {idx: label for idx, label in enumerate(labels)}

# ----------------------------
# Definir mapeo de predicciones para cambiar LABEL_0 y LABEL_1
# ----------------------------
# Mapea las predicciones del modelo a 'benigna' y 'maligna'
prediction_mapping = {
    'LABEL_0': 'benigna',
    'LABEL_1': 'maligna'
}

# ----------------------------
# Inicializar el pipeline de clasificación de imágenes
# ----------------------------
# Nota: Asegúrate de que tu dispositivo "mps" (Metal Performance Shaders) es compatible.
# Si estás usando una GPU NVIDIA, podrías cambiar a "cuda".
classifier = pipeline("image-classification", model=_checkpoint, device="mps")

# ----------------------------
# Contador para limitar el número de inferencias (opcional)
# ----------------------------
max_inferences = 100  # Número máximo de imágenes a procesar
cont = 0

# ----------------------------
# Iterar sobre las predicciones del pipeline
# ----------------------------
for i, out in enumerate(classifier(KeyDataset(dataset, "image"))):
    if cont >= max_inferences:
        break
    else:
        predicted_label = out[0]['label']  # Etiqueta predicha ('LABEL_0' o 'LABEL_1')
        predicted_score = out[0]['score']  # Confianza de la predicción
        real_label = id2label[dataset[i]['label']]  # Etiqueta real (ej. 'Benign Masses' o 'Malignant Masses')
        
        # Cambiar las predicciones 'LABEL_0' a 'benigna' y 'LABEL_1' a 'maligna'
        mapped_predicted_label = prediction_mapping.get(predicted_label, predicted_label)
        
        # Mostrar los resultados de la predicción
        print(f"Predicción: {mapped_predicted_label} - {predicted_score * 100:.2f}%, Etiqueta real: {real_label}")
        
        cont += 1

# ----------------------------
# Inferencia sobre una imagen individual (corregido)
# ----------------------------
imagen_individual = '/Volumes/m2/Memoria/Code/PMM/BeningMalignant_Masses-Preprocessing/data/processed/subset_datos/vinddsm/Benign Masses/0b873faadf6723e422e7ad7c1accc7c0_0.png'

# Verificar que la ruta de la imagen existe
if not os.path.exists(imagen_individual):
    raise FileNotFoundError(f"La imagen especificada no existe: {imagen_individual}")

# Abrir la imagen usando PIL
try:
    image = Image.open(imagen_individual).convert('RGB')  # Convertir a RGB para asegurar compatibilidad
except Exception as e:
    raise ValueError(f"No se pudo abrir la imagen. Error: {e}")

# Realizar la inferencia sobre la imagen individual
resultado_individual = classifier(image)

# Mapear las predicciones de la inferencia individual de 'LABEL_0' y 'LABEL_1' a 'benigna' y 'maligna'
mapped_resultado_individual = [
    {'label': prediction_mapping.get(item['label'], item['label']), 'score': item['score']} 
    for item in resultado_individual
]

# Mostrar los resultados de la inferencia individual
print("\nInferencia sobre una imagen individual:")
print(mapped_resultado_individual)


Predicción: maligna - 99.80%, Etiqueta real: Malignant Masses
Predicción: maligna - 95.95%, Etiqueta real: Malignant Masses
Predicción: maligna - 98.45%, Etiqueta real: Malignant Masses
Predicción: maligna - 99.70%, Etiqueta real: Malignant Masses
Predicción: maligna - 97.36%, Etiqueta real: Malignant Masses
Predicción: maligna - 88.82%, Etiqueta real: Malignant Masses
Predicción: maligna - 99.73%, Etiqueta real: Benign Masses
Predicción: maligna - 98.80%, Etiqueta real: Malignant Masses
Predicción: maligna - 97.39%, Etiqueta real: Malignant Masses
Predicción: maligna - 99.93%, Etiqueta real: Malignant Masses
Predicción: maligna - 99.78%, Etiqueta real: Malignant Masses
Predicción: maligna - 85.55%, Etiqueta real: Malignant Masses
Predicción: maligna - 99.49%, Etiqueta real: Malignant Masses
Predicción: maligna - 99.03%, Etiqueta real: Benign Masses
Predicción: maligna - 99.78%, Etiqueta real: Malignant Masses
Predicción: benigna - 99.72%, Etiqueta real: Benign Masses
Predicción: benig